## THC FeMoco compliation

In this notebook we provide an example of the CCZ cost model functionalities by attempting to reproduce results from section IV.C of [Lee et al. 2021](https://link.aps.org/doi/10.1103/PRXQuantum.2.030305) (surface code compliation of quantum phase estimation on the THC-encoded Li et al. FeMoco Hamiltonian).


### manual choice of distillation and data parameters

The physical costs can be computed by specifying manually the factories used for magic state distillation and the surface code parameters.

In [ ]:
from qualtran.surface_code.ccz2t_cost_model import get_ccz2t_costs, CCZ2TFactory
from qualtran.surface_code.multi_factory import MultiFactory
from qualtran.surface_code.data_block import SimpleDataBlock
from qualtran.surface_code.algorithm_summary import AlgorithmSummary

n_magic = AlgorithmSummary(toffoli_gates=6665400000) # pag. 26
n_data_qubits = 696 # Fig. 10 
factory = MultiFactory(base_factory=CCZ2TFactory(distillation_l1_d=19, distillation_l2_d=31),
                       n_factories=4)
data_block = SimpleDataBlock(data_d=31, routing_overhead=0.5) 

cost = get_ccz2t_costs(
    n_magic=n_magic,
    n_algo_qubits=n_data_qubits,
    phys_err=1e-3,
    cycle_time_us=1,
    factory=factory,
    data_block=data_block
)

print(f'failure probability: {cost.failure_prob:.3%}')
print(f'wall time: {cost.duration_hr/24} days') # ref: 3 days
print(f'footprint: {cost.footprint*1e-6:.2f} million qubits') # ref: 4 million qubits 

### Grid search for distillation and data code distances

Alternatively, qualtran provides a function to perform grid search over factories and data blocks, optimizing the cost (in terms of spacetime volume, total time or any other given ordering).

In [ ]:
from qualtran.surface_code.ccz2t_cost_model import get_ccz2t_costs_from_grid_search, \
                                                   iter_ccz2t_factories

best_cost, best_factory, best_data_block = get_ccz2t_costs_from_grid_search(
    n_magic=n_magic,
    n_algo_qubits=n_data_qubits,
    error_budget=1e-2,
    phys_err=1e-3,
    factory_iter=iter_ccz2t_factories(n_factories=4), # use 4 CCZ factories in parallel
    cost_function=(lambda pc: pc.duration_hr) # optimize over total time
)

In [ ]:
distillation_error = best_factory.distillation_error(n_magic, phys_err=1e-3)
data_error = best_data_block.data_error(
    n_algo_qubits=n_data_qubits, n_cycles=best_factory.n_cycles(n_magic), phys_err=1e-3
)

print(f"distillation error: {distillation_error:.3%}")  # ref: 0.1% per 1e10 Toffolis
print(f"data error: {data_error:.3%}")
print(f"total failure probability: {best_cost.failure_prob:.3%}")
print(f"wall time: {best_cost.duration_hr / 24} days")  # ref: 3 days
print(f"footprint: {best_cost.footprint * 1e-6:.2f} million qubits")  # ref: 4 million qubits

The optimization result does not match the THC paper results exactly. 
The cost model used is similar, but the results from the THC paper include a detailed study of the lattice surgery layout. 
Here, instead, we just assume a 50% space overhead for routing and we compute the time as only limited by the CCZ generation rate.
The shorter runtime, at a cost of an increased number of physical qubits, might be due to this.